In [7]:
import os
import time
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation

from utils import *

In [8]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [ ]:
drivers = """
model_drivers:
    iiwa: !IiwaDriver
        control_mode: position_only
        hand_model_name: wsg
    wsg: !SchunkWsgDriver

cameras:
    camera0:
        name: camera0
        depth: true
        X_PB:
            base_frame: camera0::base

    camera1:
        name: camera1
        depth: true
        X_PB:
            base_frame: camera1::base

    camera2:
        name: camera2
        depth: true
        X_PB:
            base_frame: camera2::base
"""

directives = f"""
directives:
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [ 1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0
        X_PC:
            translation: [0, -0.5, 0]
            rotation: !Rpy {{ deg: [0, 0, 180] }}
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.09]
            rotation: !Rpy {{deg: [90, 0, 90]}}
    - add_model:
        name: floor
        file: package://stackbot/floor.sdf
    - add_weld:
        parent: world
        child: floor::floor_link
        X_PC:
            translation: [0, 0, -0.05]
    - add_model:
        name: platform
        file: package://stackbot/platform.sdf
    - add_weld:
        parent: world
        child: platform::platform_link
"""

block_numbers = np.random.choice(range(11), size=np.random.randint(4, 7), replace=False)
print("This scenario uses blocks:", block_numbers)
blocks = [f"block{i}" for i in block_numbers]

block_sdfs = [f"""
    - add_model:
        name: {block}
        file: package://stackbot/{block}.sdf
""" for block in blocks]

scenario_yaml = "\n".join([directives] + block_sdfs)

This scenario uses blocks: [10  2  0  7]


In [21]:
# Add cameras to the scenario
scenario_yaml += f"""
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [-120.0, 0.0, 180.0]}}
            translation: [0, 0.8, 0.5]

    - add_model:
        name: camera0
        file: package://manipulation/camera_box.sdf

    - add_weld:
        parent: camera0_origin
        child: camera0::base

    - add_frame:
        name: camera1_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [-125, 0.0, 90.0]}}
            translation: [0.8, 0.1, 0.5]

    - add_model:
        name: camera1
        file: package://manipulation/camera_box.sdf

    - add_weld:
        parent: camera1_origin
        child: camera1::base

    - add_frame:
        name: camera2_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [-120.0, 0.0, -90.0]}}
            translation: [-0.8, 0.1, 0.5]

    - add_model:
        name: camera2
        file: package://manipulation/camera_box.sdf

    - add_weld:
        parent: camera2_origin
        child: camera2::base
"""
# save so we can add drivers
with open("directives/directives.dmd.yaml", "w") as f:
        f.write(scenario_yaml)

In [22]:
scenario_with_drivers_yaml = """
directives:
    - add_directives:
        file: file://{os.getcwd()}/directives/bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml
"""
scenario_with_drivers_yaml += drivers

In [23]:
station = MakeHardwareStation(LoadScenario(data=scenario_yaml), meshcat=meshcat, package_xmls=["../assets/models/package.xml"])
plant = station.GetSubsystemByName("plant")
builder = DiagramBuilder()
builder.AddSystem(station)

# connect etc.

diagram = builder.Build()

In [24]:
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)

randomize_blocks(blocks, plant, plant_context)

In [25]:
simulator = Simulator(diagram, diagram_context)

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5)
meshcat.StopRecording()
meshcat.PublishRecording()